In [ ]:
# #############################################################################
# Importing necessary libraries

import numpy as np
import pandas as pd
import requests as requests
import json
from pathlib import Path
from m3inference import M3Inference
from collections import OrderedDict
import pprint
import urllib.request
import random
import re
import json
import os
from PIL import Image
import csv 
import sys
import string 
import zipfile as zf
from sklearn import metrics

In [ ]:
def extract_files(path, file):# Expanding the zip files
    files = zf.ZipFile(path+file, 'r')
    files.extractall(path+'a')
    files.close()

# Convert all the images in the RGB formate
def preprocess_images(file_name, height, width):
    im = Image.open(file_name)
    resized_im = im.resize((height, width))
    resized_im.convert('RGB').save(file_name)
        

# Generate random numbers
def randomNumber(Numberlength):
    for x in range(Numberlength):
        number= random.randint(1,10001)
    return number

# Running the M3 inference and getting the gender
def Mi_inference(Imagelocation,Imagename):
    config = Path(Imagelocation)
    value=randomNumber(10)

    if config.is_file():                               # here we passing parameters in random both string and number
        data= {
            "description":"", 
            "id": value,
            "img_path": Imagelocation, 
            "lang": "en", 
            "name": "", 
            "screen_name": ""
        }
        Json_file=path+'data.jsonl'        # Path to create the Json file
        with open(Json_file, 'w') as json_file:
            json.dump(data, json_file) 
        try:                                            # Here Based on values in output will select the geneder
            m3 = M3Inference()                          # see docstring for details
            pred = m3.infer(Json_file)                  # also see docstring for details
            disc=pprint.pprint(pred)
            main=dict(pred).get(value)
            main_gender=main.get('gender')
            main_gender_female=main_gender.get('female')
            main_gender_male=main_gender.get('male')
            if main_gender_female > main_gender_male :
                Gender='F'
            elif main_gender_female < main_gender_male :
                Gender='M'
            else:
                Gender='N'
        except urllib.error.HTTPError as e:
            print (e)
    else:
        Gender='NI'
    return Gender

# VARIABLES TO CHANGE:
path = '../../GIGIDL/Data/'
file = 'Male.zip'
true_gender = 'M'

extract_files(path, file)

# Running the imageson M3 and getting the output
with open(path + file.split('.')[0] + '.csv', 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Imagename','G_Gender','M_Gender'])
    files_path = path + 'a/' + file.split('.')[0]+'/'
    for Imagename in os.listdir(files_path):                        # Get the images from directory
        try:
            Image_location=files_path+Imagename # Get the image loacation of each image
            preprocess_images(Image_location, 224, 224)
            Mi_gender=Mi_inference(Image_location,Imagename)
            wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
            wr.writerow([Imagename,true_gender,Mi_gender])
        except:
            print("Oops!",sys.exc_info()[0],"occured.")
            print()


In [ ]:
file_f = '../Predicted_Results/blurry_f.csv'
file_m = '../Predicted_Results/blurry_m.csv'

female_all= pd.read_csv(file_f)
male_all= pd.read_csv(file_m)
full_data = pd.concat([female_all, male_all])# 
# True values
y_true = GSfull_data['G_Gender']
# Predicted values
y_pred = GSfull_data['M_Gender']

In [ ]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

#printing the metrics
metrics=metrics.classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics['weighted avg']['precision'],3))
#Recall
print('Recall:',round(metrics['weighted avg']['recall'],3))
#F1-score
print('F1-score:',round(metrics['weighted avg']['f1-score'],3))
#accuracy
print('Accuracy:',round(metrics['accuracy'],3))